### Authenticate

Get the client id and secret from env vars for prod.
In dev we can create a new client each time


In [ ]:
import { registerSystem } from '../v1-to-v2-data-migration/helpers/gqlHandlers.ts'
import {
  authenticate,
  getTokenForSystemClient,
} from '../v1-to-v2-data-migration/helpers/authentication.ts'
import {
  CLIENT_ID,
  CLIENT_SECRET,
} from '../v1-to-v2-data-migration/helpers/vars.ts'
import {
  ADMIN_USERNAME,
  ADMIN_PASSWORD,
} from '../v1-to-v2-data-migration/helpers/vars.ts'

let clientId: string | null = null
let clientSecret: string | null = null

if (CLIENT_ID && CLIENT_SECRET) {
  clientId = CLIENT_ID
  clientSecret = CLIENT_SECRET
} else {
  // For dev mode
  const adminToken = await authenticate(ADMIN_USERNAME, ADMIN_PASSWORD)
  const systemRegistration = await registerSystem(adminToken)

  clientId = systemRegistration.data.registerSystem.system.clientId
  clientSecret = systemRegistration.data.registerSystem.clientSecret
}

const sysToken = await getTokenForSystemClient(clientId, clientSecret)
sysToken


### Fetch data from CSVs


In [ ]:
import { csvToJson } from './helpers/csvHelpers.ts'
import { CsvFields } from './helpers/csvTypes.ts'

const pathToBirthCsv = './sourceData/Birth_Register.csv'
const pathToDeathCsv = './sourceData/Death_Register.csv'
const pathToMarriageCsv = './sourceData/Marriage_Register.csv'
const pathToAdoptionCsv = './sourceData/Adoption_Register.csv'
const pathToDeedpollCsv = './sourceData/Deedpoll.csv'

const csvData: CsvFields = {
  birth: await csvToJson(pathToBirthCsv),
  death: await csvToJson(pathToDeathCsv),
  marriage: await csvToJson(pathToMarriageCsv),
  adoption: await csvToJson(pathToAdoptionCsv),
  deedpoll: await csvToJson(pathToDeedpollCsv),
}


In [ ]:
import { GATEWAY } from '../v1-to-v2-data-migration/helpers/routes.ts'
import { Country } from './helpers/addressConfig.ts'
import { readJsonFile } from './helpers/csvHelpers.ts'
import { LocationMap } from './helpers/types.ts'

export const getLocations = async (
  token: string,
  type: 'ADMIN_STRUCTURE' | 'HEALTH_FACILITY',
) => {
  const response = await fetch(
    `${GATEWAY}/location?type=${type}&_count=0&status=active`,
    {
      method: 'GET',
      headers: {
        'Content-Type': 'application/json',
        Authorization: `Bearer ${token}`,
      },
    },
  )
  if (!response.ok) {
    throw new Error(`Sync Locations failed: ${response.statusText}`)
  }
  return response
}

const locationsRes = await getLocations(sysToken, 'ADMIN_STRUCTURE')
const healthFacilitiesRes = await getLocations(sysToken, 'HEALTH_FACILITY')
const fhirLocations = await locationsRes.json()
const fhirHealthFacilities = await healthFacilitiesRes.json()

const locationCodes = fhirLocations.entry
  .concat(fhirHealthFacilities.entry)
  .map((loc: any) => ({
    id: loc.resource.id,
    name: loc.resource.name,
    code:
      loc.resource.identifier[0]?.value
        .replace('ADMIN_STRUCTURE_', '')
        .replace('HEALTH_FACILITY_', '') || null,
  }))

const intl: Record<string, { country: Country; town: string }> = readJsonFile(
  './lookupMappings/generated/intl.json',
)
const local: Record<string, string> = readJsonFile(
  './lookupMappings/generated/local.json',
)
const facility: Record<string, string> = readJsonFile(
  './lookupMappings/generated/facility.json',
)

const locationMap: LocationMap[] = []

Object.entries(intl).forEach(([key, value]) => {
  locationMap.push({
    id: '',
    name: key,
    country: value.country,
    intlTown: value.town.trim(),
  })
})

Object.entries(local).forEach(([key, value]) => {
  locationMap.push({
    id: '',
    map: value,
    name: key,
  })
})

Object.entries(facility).forEach(([key, value]) => {
  locationMap.push({
    id: '',
    facilityCode: value,
    name: key,
  })
})

const updatedLocationsMap = locationMap.map((loc: any) => ({
  ...loc,
  id:
    locationCodes.find(
      (l: any) => l.code === loc.map || l.code === loc.facilityCode,
    )?.id || null,
}))
updatedLocationsMap


### Run birth related migrations





In [ ]:
import { birthResolver, birthMetaData } from './mappings/birthResolver.ts'
import { transform } from './helpers/transform.ts'
import { bulkImport } from '../v1-to-v2-data-migration/helpers/gqlHandlers.ts'
import {
  batch,
  getIndexErrors,
} from '../v1-to-v2-data-migration/helpers/utils.ts'
import {
  Collisions,
  generateRegistrationNumber,
  Hashes,
} from './helpers/generators.ts'
import { CrvsEvent } from './helpers/types.ts'
import { BirthMetaData, BirthResolver } from './helpers/birthTypes.ts'
import { toCrvsDate, toLegacy } from './helpers/resolverHelpers.ts'
import { nameChangeResolver } from './mappings/nameChangeResolver.ts'
import { NameChangeResolver } from './helpers/nameChangeTypes.ts'

function nonNullObjectKeys(obj: Record<string, any>) {
  return Object.fromEntries(
    Object.entries(obj).filter(
      ([_, value]) => value !== null && value !== undefined && value !== '',
    ),
  )
}

const events: Array<CrvsEvent> = []

csvData.birth
  .filter((x) => x.BIRTH_REF)
  .forEach((birth) => {
    const declaration: Partial<Record<keyof BirthResolver, any>> = {}

    Object.entries(birthResolver).forEach(([eventField, dataField]) => {
      if (typeof dataField === 'function') {
        const data = dataField(birth, csvData, updatedLocationsMap)
        declaration[eventField as keyof BirthResolver] = data
      }
    })

    const meta: Partial<Record<keyof typeof birthMetaData, any>> = {}
    Object.entries(birthMetaData).forEach(([metaField, dataField]) => {
      const data = dataField(birth, csvData, updatedLocationsMap)
      meta[metaField as keyof BirthMetaData] = data
    })

    const user = '5d53c989-89a3-40d2-bbc1-fe1d3a9aba86'
    const role = 'REGISTRAR' // Probably ok to hardcode this

    const locationId = locationCodes.find(
      (loc: any) => loc.code === meta.locationCode,
    )?.id

    const event = transform(
      nonNullObjectKeys(declaration),
      'birth',
      meta.registrationDate || new Date().toISOString(),
      user, //TODO change to lookup meta.registrar,
      role,
      locationId,
      birth.BIRTH_REF,
      toLegacy(birth.BIRTH_REF, 'birth'),
    )

    events.push(event)

    const nameChanges = csvData.deedpoll
      .filter(
        (deed) =>
          deed.BIRTH_REF.toUpperCase() === birth.BIRTH_REF.toUpperCase(),
      )
      .sort(
        (a, b) =>
          new Date(toCrvsDate(a.DATE)).getTime() -
          new Date(toCrvsDate(b.DATE)).getTime(),
      )
    if (nameChanges.length > 0) {
      const nameChangeDeclaration: Partial<
        Record<keyof NameChangeResolver, any>
      > = {}

      nameChanges.forEach((nc) => {
        Object.entries(nameChangeResolver).forEach(
          ([eventField, dataField]) => {
            if (typeof dataField === 'function') {
              const data = dataField(nc, birth, nameChanges, locationMap)
              nameChangeDeclaration[eventField as keyof NameChangeResolver] =
                data
            }
          },
        )

        const changeEvent = transform(
          nonNullObjectKeys(nameChangeDeclaration),
          'name-change',
          meta.registrationDate || new Date().toISOString(),
          user, //TODO change to lookup meta.registrar,
          role,
          locationId,
          birth.BIRTH_REF,
          generateRegistrationNumber(
            meta.locationCode || '',
            meta.registrationDate || new Date().toISOString(),
            'birth',
            nc.DP_REF,
          ),
        )
        events.push(changeEvent)
      })
    }
  })

const batches = batch(events, 1000)

for (const batch of [batches[1]]) {
  const res = await bulkImport(batch, sysToken)
  console.log(res)
  const errors = getIndexErrors(res)
  if (errors) {
    console.error('Errors during bulk import', errors)
  }
}


In [ ]:
import { DeathResolver } from './helpers/deathTypes.ts'
import { deathResolver, deathMetaData } from './mappings/deathResolver.ts'

const deathEvents: Array<CrvsEvent> = []

csvData.death
  .filter((x) => x.DEATH_NUMBER)
  .forEach((death) => {
    const declaration: Partial<Record<keyof DeathResolver, any>> = {}

    Object.entries(deathResolver).forEach(([eventField, dataField]) => {
      if (typeof dataField === 'function') {
        const data = dataField(death, csvData, updatedLocationsMap)
        declaration[eventField as keyof DeathResolver] = data
      }
    })

    const meta: Partial<Record<keyof typeof deathMetaData, any>> = {}
    Object.entries(deathMetaData).forEach(([metaField, dataField]) => {
      const data = dataField(death, csvData, updatedLocationsMap)
      meta[metaField as keyof typeof deathMetaData] = data
    })

    const user = '5d53c989-89a3-40d2-bbc1-fe1d3a9aba86'
    const role = 'REGISTRAR' // Probably ok to hardcode this

    const locationId = locationCodes.find(
      (loc: any) => loc.code === meta.locationCode,
    )?.id

    const event = transform(
      nonNullObjectKeys(declaration),
      'death',
      meta.registrationDate || new Date().toISOString(),
      user, //TODO change to lookup meta.registrar,
      role,
      locationId,
      death.DEATH_NUMBER,
      toLegacy(death.DEATH_NUMBER, 'death'),
    )

    deathEvents.push(event)
  })

const deathBatches = batch(deathEvents, 1000)

for (const batch of [deathBatches[1]]) {
  const res = await bulkImport(batch, sysToken)
  console.log(res)
  const errors = getIndexErrors(res)
  if (errors) {
    console.error('Errors during bulk import', errors)
  }
}


In [ ]:
import { reindex } from '../v1-to-v2-data-migration/helpers/gqlHandlers.ts'

const reindexResponse = await reindex(sysToken)
reindexResponse


### Output results


In [ ]:
console.log('🍞 Declarations succesfully migrated')
